# 第3部: 量子アニーリング編 - 未来の計算機で究極のレシピを探求！
# ステップ11: 量子アニーリングとFixstars Amplifyの紹介 - 量子の力で最適化！

In [3]:
# Amplify SDKのインストール:
!pip install --upgrade amplify


# ステップ12: Amplifyで問題を定義し、量子アニーリングで解く - 量子のレシピに挑戦！

In [6]:
# 必要なライブラリのインポート:
from amplify import BinaryPoly, BinarySymbolGenerator, Solver, sum_poly
from amplify.client import FixstarsClient
from amplify.constraint import equal_to, less_equal, greater_equal
import numpy as np

# アクセストークンの設定 (超重要！):
# @title
token = "あなたのトークン" # トークンを入力
# token = "AE/****" # トークンを入力

# 各種データの定義:
# Creates a list of the Ingredients
Ingredients = ["Ebi", "Tatukuri", "Kazunoko", "Kuromame", "Tamago"]

# A dictionary of the costs of each of the Ingredients is created
tanka_costs = {
    "Ebi": 3.0,
    "Tatukuri": 5.0,
    "Kazunoko": 4.0,
    "Kuromame": 2.0,
    "Tamago": 1.5,
}

min_unit = {
    "Ebi": 15.0,
    "Tatukuri": 2.0,
    "Kazunoko": 25.0,
    "Kuromame": 3.0,
    "Tamago": 30.0,
}

# A dictionary of the tanpakushitu percent in each of the Ingredients is created
tanpakushitu = {
    "Ebi": 0.18,
    "Tatukuri": 0.7,
    "Kazunoko": 0.29,
    "Kuromame": 0.09,
    "Tamago": 0.1,
}

shibou = {
    "Ebi": 0.01,
    "Tatukuri": 0.05,
    "Kazunoko": 0.02,
    "Kuromame": 0.02,
    "Tamago": 0.08,
}

tansuikabutu = {
    "Ebi": 0.0,
    "Tatukuri": 0.01,
    "Kazunoko": 0.0,
    "Kuromame": 0.54,
    "Tamago": 0.04,
}

enbun = {
    "Ebi": 0.007,
    "Tatukuri": 0.015,
    "Kazunoko": 0.06,
    "Kuromame": 0.002,
    "Tamago": 0.012,
}

# 決定変数の定義 (Amplify流):
from amplify import VariableGenerator

# Number of ingredients
n = len(Ingredients)

gen = VariableGenerator()
# 今回は整数変数を扱いますが、Amplifyの基本的な問題形式(QUBO)はバイナリ変数です。
# ここでは整数変数を扱うために bounds を指定しています。
# 量子アニーリングで整数を扱う場合、内部的にはバイナリ変数にエンコードされます。
ingredient_vars = gen.array("Integer", shape=np.array([n]), bounds=(0, 100))

# コスト関数（目的関数）の定義:
# Define the cost function
cost_function = sum_poly( # sum_poly はAmplifyの多項式の和を取る関数
    tanka_costs[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i]
    for i in range(n)
)

# 制約条件の定義:
# Define the constraints
constraints = [
    greater_equal( # タンパク質 >= 100
        sum_poly(tanpakushitu[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i] for i in range(n)),
        100.0,
    ),
    less_equal( # 脂肪 <= 50
        sum_poly(shibou[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i] for i in range(n)),
        50.0,
    ),
    # (炭水化物、塩分の制約も同様)
    less_equal(
        sum(tansuikabutu[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i] for i in range(n)),
        50.0,
    ),
    less_equal(
        sum(enbun[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_vars[i] for i in range(n)),
        10.0,
    ),

    # (各具材の個数の下限・上限の制約も同様に追加)
    # 例: エビの個数 >= 2
    greater_equal(ingredient_vars[0], 2), # Ingredientsリストの0番目がEbi
    # 例: エビの個数 <= 10
    less_equal(ingredient_vars[0], 10),
    # ... 他の具材も同様
    greater_equal(ingredient_vars[1], 5), # Tatukuri
    less_equal(ingredient_vars[1], 30),

    greater_equal(ingredient_vars[2], 2), # Kazunoko
    less_equal(ingredient_vars[2], 10),

    greater_equal(ingredient_vars[3], 5), # Kuromame
    less_equal(ingredient_vars[3], 30),

    greater_equal(ingredient_vars[4], 2), # Tamago
    less_equal(ingredient_vars[4], 10),

]

# 問題の構築 (コスト関数 + 制約):
era = 5.0 # 制約の重み（ペナルティの強さ）。調整が必要なハイパーパラメータ。
problem = cost_function + era * sum_poly(constraints) # sum_polyで制約のリストを一つの多項式に

# クライアントの設定と問題の解決:
# Set up the client
client = FixstarsClient()
client.token = token  # ここで設定したトークンが使われる
client.parameters.timeout = 10000  # タイムアウトを10秒に設定 (ミリ秒単位)

# Solve the problem
solver = Solver(client) # クライアントを使ってソルバーを初期化
result = solver.solve(problem) # 問題を解く

# 結果の取り出し方がPuLPと少し異なります。
if len(result) == 0: # 解が見つからなかった場合
    print("No solution found")
else:
    solution = result[0].values # 最良解の値を取得
    # ingredient_vars を使って、どの変数がどの値になったかを取り出す
    ingredient_values = [solution[var] for var in ingredient_vars]
    for i, ingredient in enumerate(Ingredients):
        print(f"{ingredient}: {ingredient_values[i]}")
    # (栄養素の計算と総費用の計算・表示はPuLP編とほぼ同様)
    # Calculate nutritional values
    tanpaku = sum(
        tanpakushitu[ingredient] * min_unit[ingredient] * ingredient_values[i] # Use ingredient_values[i]
        for i, ingredient in enumerate(Ingredients)
    )
    shi = sum(
        shibou[ingredient] * min_unit[ingredient] * ingredient_values[i] # Use ingredient_values[i]
        for i, ingredient in enumerate(Ingredients)
    )
    tan = sum(
        tansuikabutu[ingredient] * min_unit[ingredient] * ingredient_values[i] # Use ingredient_values[i]
        for i, ingredient in enumerate(Ingredients)
    )
    en = sum(
        enbun[ingredient] * min_unit[ingredient] * ingredient_values[i] # Use ingredient_values[i]
        for i, ingredient in enumerate(Ingredients)
    )
    print()
    print(f"tanpakushitu = {tanpaku}")
    print(f"shibou = {shi}")
    print(f"tansuikabutu = {tan}")
    print(f"enbun = {en}")
    print()
    total_cost = sum(
        tanka_costs[Ingredients[i]] * min_unit[Ingredients[i]] * ingredient_values[i] # Use ingredient_values[i]
        for i in range(n)
    ) # Calculate the total cost here
    print(f"Total_Cost_of_Ingredients = {total_cost}") # Print the calculated total cost
    print()






Ebi: 4.0
Tatukuri: 30.0
Kazunoko: 5.0
Kuromame: 8.0
Tamago: 3.0

tanpakushitu = 100.20999999999998
shibou = 13.78
tansuikabutu = 17.16
enbun = 9.948

Total_Cost_of_Ingredients = 1163.0



/tmp/ipython-input-6-8782838.py:76: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ingredient_vars = gen.array("Integer", shape=np.array([n]), bounds=(0, 100))
/tmp/ipython-input-6-8782838.py:80: DeprecationWarning: sum_poly is deprecated since amplify v1.0.0 and will no longer support in the near future.
Use sum() instead. Please see the migration guide for details: https://amplify.fixstars.com/docs/amplify/v1/migration.html
  cost_function = sum_poly( # sum_poly はAmplifyの多項式の和を取る関数
/tmp/ipython-input-6-8782838.py:89: DeprecationWarning: sum_poly is deprecated since amplify v1.0.0 and will no longer support in the near future.
Use sum() instead. Please see the migration guide for details: https://amplify.fixstars.com/docs/amplify/v1/migration.html
  sum_poly(tanpakushitu[Ingredients[i]] * min_unit[Ingredients[i]

# 結果の比較と考察:
## Amplify（量子アニーリング）で得られた結果は、PuLPやExcelソルバーの結果と比べてどうでしたか？
## もし結果が異なっていたら、それはなぜだと思いますか？（ヒント：量子アニーリングの特性、era の値、計算時間など）
## 量子アニーリングを使うメリットやデメリットは何だと思いますか？